In [5]:
import customtkinter as ctk

class Node:
    """
    Merepresentasikan sebuah simpul (node) dalam N-ary tree.
    Setiap node memiliki nilai, daftar anak, dan referensi ke induknya.
    """
    def __init__(self, value, parent=None):
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child_node):
        """Menambahkan child_node sebagai anak dari node ini."""
        child_node.parent = self
        self.children.append(child_node)

    def get_tree_representation(self):
        """Menghasilkan representasi string dari subtree yang berakar di node ini."""
        if not self:
            return ""
        
        lines = [str(self.value)] # Nilai root/node saat ini
        
        children_count = len(self.children)
        for i, child in enumerate(self.children):
            # Memanggil helper untuk menambahkan representasi anak-anaknya
            child._append_subtree_lines(lines, "", i == children_count - 1)
        return "\n".join(lines)

    def _append_subtree_lines(self, lines, prefix, is_last_child):
        """
        Helper rekursif untuk membangun representasi string dari subtree.
        lines: list string yang sedang dibangun.
        prefix: awalan untuk baris saat ini (terdiri dari spasi dan '│').
        is_last_child: boolean apakah node ini adalah anak terakhir dari induknya.
        """
        lines.append(prefix + ("└── " if is_last_child else "├── ") + str(self.value))
        
        children_count = len(self.children)
        for i, child in enumerate(self.children):
            new_prefix = prefix + ("    " if is_last_child else "│   ")
            child._append_subtree_lines(lines, new_prefix, i == children_count - 1)

class N_aryTree:
    """
    Merepresentasikan struktur data N-ary Tree.
    """
    def __init__(self):
        self.root = None

    def add_root(self, value):
        """Menambahkan root ke tree. Hanya jika tree masih kosong."""
        if self.root is None:
            self.root = Node(value)
            return self.root
        else:
            # Sebaiknya ditangani di GUI untuk tidak memanggil ini jika root sudah ada
            print("Root sudah ada.") 
            return None

    def _find_node_recursive(self, current_node, target_value):
        """Helper rekursif untuk mencari node dengan target_value (DFS)."""
        if current_node is None:
            return None
        if current_node.value == target_value:
            return current_node
        for child in current_node.children:
            found_node = self._find_node_recursive(child, target_value)
            if found_node:
                return found_node
        return None

    def find_node(self, target_value):
        """Mencari node dengan target_value di tree."""
        if not self.root:
            return None
        return self._find_node_recursive(self.root, target_value)

    def add_node(self, parent_value, child_value):
        """Menambahkan child_value sebagai anak dari parent_value."""
        if self.root is None:
            # print("Tree kosong. Tambahkan root terlebih dahulu.") # Ditangani GUI
            return None

        parent_node = self.find_node(parent_value)
        if parent_node:
            # Opsional: Cek jika anak dengan nilai yang sama sudah ada
            # for existing_child in parent_node.children:
            #     if existing_child.value == child_value:
            #         print(f"Node '{child_value}' sudah menjadi anak dari '{parent_value}'.")
            #         return None
            child_node = Node(child_value, parent=parent_node)
            parent_node.add_child(child_node)
            return child_node
        else:
            # print(f"Parent node dengan nilai '{parent_value}' tidak ditemukan.") # Ditangani GUI
            return None

    def add_leaf(self, parent_value, leaf_value):
        """
        Menambahkan leaf_value sebagai anak dari parent_value.
        Secara fungsional sama dengan add_node.
        """
        return self.add_node(parent_value, leaf_value)

    def get_tree_string(self):
        """Mengembalikan representasi string dari keseluruhan tree."""
        if self.root:
            return self.root.get_tree_representation()
        else:
            return "Tree kosong."

class App(ctk.CTk):
    def __init__(self, tree_instance):
        super().__init__()
        self.tree = tree_instance

        self.title("N-ary Tree GUI (CustomTkinter)")
        self.geometry("600x700")

        ctk.set_appearance_mode("System") 
        ctk.set_default_color_theme("blue")

        # Frame untuk menambah root
        self.root_frame = ctk.CTkFrame(self)
        self.root_frame.pack(pady=10, padx=10, fill="x")

        ctk.CTkLabel(self.root_frame, text="Nilai Root:").pack(side="left", padx=5)
        self.root_value_entry = ctk.CTkEntry(self.root_frame, placeholder_text="Masukkan nilai root")
        self.root_value_entry.pack(side="left", padx=5, expand=True, fill="x")
        self.add_root_button = ctk.CTkButton(self.root_frame, text="Tambah Root", command=self.gui_add_root)
        self.add_root_button.pack(side="left", padx=5)

        # Frame untuk menambah node/leaf
        self.node_frame = ctk.CTkFrame(self)
        self.node_frame.pack(pady=10, padx=10, fill="x")

        ctk.CTkLabel(self.node_frame, text="Nilai Parent:").pack(side="left", padx=5)
        self.parent_value_entry = ctk.CTkEntry(self.node_frame, placeholder_text="Masukkan nilai parent")
        self.parent_value_entry.pack(side="left", padx=5, expand=True, fill="x")

        ctk.CTkLabel(self.node_frame, text="Nilai Child:").pack(side="left", padx=5)
        self.child_value_entry = ctk.CTkEntry(self.node_frame, placeholder_text="Masukkan nilai child")
        self.child_value_entry.pack(side="left", padx=5, expand=True, fill="x")

        self.add_node_button = ctk.CTkButton(self.node_frame, text="Tambah Node/Leaf", command=self.gui_add_node)
        self.add_node_button.pack(side="left", padx=5)

        # Area tampilan tree
        self.tree_display_textbox = ctk.CTkTextbox(self, height=300, width=580, wrap="none") # wrap="none" untuk scroll horizontal jika perlu
        self.tree_display_textbox.pack(pady=10, padx=10, fill="both", expand=True)
        self.tree_display_textbox.configure(state="disabled") 

        # Status bar
        self.status_label = ctk.CTkLabel(self, text="Status: Siap", anchor="w")
        self.status_label.pack(pady=5, padx=10, fill="x")

        self.update_tree_display()

    def gui_add_root(self):
        value = self.root_value_entry.get()
        if not value:
            self.update_status("Nilai root tidak boleh kosong.")
            return
        
        if self.tree.root is not None:
            self.update_status(f"Root sudah ada: '{self.tree.root.value}'. Tidak bisa menambah root lagi.")
            return

        self.tree.add_root(value)
        self.update_status(f"Root '{value}' berhasil ditambahkan.")
        self.update_tree_display()
        self.root_value_entry.delete(0, "end")

    def gui_add_node(self):
        parent_value = self.parent_value_entry.get()
        child_value = self.child_value_entry.get()

        if not parent_value or not child_value:
            self.update_status("Nilai parent dan child tidak boleh kosong.")
            return

        if self.tree.root is None:
            self.update_status("Tree kosong. Tambahkan root terlebih dahulu.")
            return

        node_added = self.tree.add_node(parent_value, child_value)
        if node_added:
            self.update_status(f"Node '{child_value}' berhasil ditambahkan ke parent '{parent_value}'.")
            self.update_tree_display()
            self.parent_value_entry.delete(0, "end")
            self.child_value_entry.delete(0, "end")
        else:
            if not self.tree.find_node(parent_value): # Cek spesifik kenapa gagal
                 self.update_status(f"Parent node '{parent_value}' tidak ditemukan.")
            else: # Mungkin ada alasan lain jika ada validasi tambahan di add_node
                self.update_status(f"Gagal menambahkan node '{child_value}'.")

    def update_tree_display(self):
        self.tree_display_textbox.configure(state="normal")
        self.tree_display_textbox.delete("1.0", "end")
        tree_str = self.tree.get_tree_string()
        self.tree_display_textbox.insert("1.0", tree_str)
        self.tree_display_textbox.configure(state="disabled")

    def update_status(self, message):
        self.status_label.configure(text=f"Status: {message}")
        print(f"GUI Status: {message}") # Juga print ke konsol untuk debug

if __name__ == "__main__":
    my_tree_instance = N_aryTree()
    app = App(my_tree_instance)
    app.mainloop()

GUI Status: Root 'fnb' berhasil ditambahkan.
GUI Status: Node 'makanan' berhasil ditambahkan ke parent 'fnb'.
GUI Status: Node 'minuman' berhasil ditambahkan ke parent 'fnb'.
GUI Status: Node 'minuman' berhasil ditambahkan ke parent 'makanan'.
